# Data reduction for Amor using `scipp`

Herein, we will look in detail at the reduction of data collected from the Amor instrument using `scipp` ([scipp.github.io](https://scipp.github.io)), giving information about the data reduction steps used at Amor. 
All of the steps given here, are implemented in the `AmorData` class ([scipp.github.io/ess/instruments/amor/amor_data](https://scipp.github.io/ess/instruments/amor/amor_data.html)) and it is shown how easily they may be accessed from this [simple reduction notebook](https://scipp.github.io/ess-notebooks/reflectometry/amor_reduction.html).

Let's start by importing the modules necessary for this notebook and loading the data, where [sample.nxs](https://github.com/scipp/ess-notebooks-data/raw/main/ess/amor/sample.nxs) is the experimental data file of interest and [reference.nxs](https://github.com/scipp/ess-notebooks-data/raw/main/ess/amor/references.nxs) is the reference measurement of the neutron supermirror.

In [ ]:
import os
import dataconfig
import numpy as np
from scipy.special import erf
import scipp as sc
import scippneutron as scn
from ess.reflectometry.data import ReflData
from ess.reflectometry import HDM, G_ACC
from ess.amor.amor_data import AmorData

local_data_path = os.path.join('ess', 'amor')
data_dir = os.path.join(dataconfig.data_root, local_data_path)
data_file = os.path.join(data_dir, 'sample.nxs')
reference_file = os.path.join(data_dir, 'reference.nxs')

## Data Storage

The data collected from the Amor instrument is written to a NeXus file, which can be read into `scipp` was follows.

In [ ]:
data = scn.load_nexus(data_file)

In [ ]:
data

The data is stored in a binned structure, where each of the bin is a pixel on the detector containing each neutron event that was measured there. 
For convinence, we will store this `data` in the `ReflData` class, which assigns values of 1 for the variances of each event (following Poisson statistics), and enables easy visualisation of all of the neutron events. 

In [ ]:
refl = ReflData(data)

In [ ]:
refl.data.events

It is also possible to visualise the detector, with the neutron events shown in the relevant pixels. 

In [ ]:
scn.instrument_view(refl.data)

## Time-of-flight correction

The time-of-flight values present in the `refl` object above is taken from the `event_time_offset`, $t_{\text{eto}}$.
Therefore, it is necessary to account for an offset due to the pulse generation. 
This correction is performed by considering half of the reciprocal rotational velocity of the chopper, $\tau$ and the phase offset between the chopper pulse and the time-of-flight $0$, $\phi_{\text{chopper}}$, 

$$
t_{\text{offset}} = \tau \phi_{\text{chopper}}.
$$

The time-of-flight is then found as, 

$$
t = [(t_{\text{eto}} - t_{\text{cut}} + \tau) \;\text{mod}\; \tau] + t_{\text{cut}} + t_{\text{offset}},
$$

where, $\text{mod}$ is the remainder between the two values and $t_{\text{cut}}$ is found from the minimum wavelength, $\lambda$, to be used. 
In order to perform this conversion, first we convert the time-of-flight coordinate in the event data to microseconds (this is for clarity, due to the commonality of the unit, though not strictly necessary). 

In [ ]:
t_eto = sc.to_unit(refl.data.events.coords['tof'], 'us')
t_eto

First, $t_{\text{offset}}$ can be found. 

In [ ]:
chopper_speed = 20 / 3 * 1e-6 / sc.units.us
tau = sc.to_unit(1 / (2 * chopper_speed), 'us')
chopper_phase = -8.0 * sc.units.deg
t_offset = sc.to_unit(tau * chopper_phase / (180.0 * sc.units.deg), 'us')
t_offset

And then $t_{\text{cut}}$ from the minimum wavelength value, where `HDM` is the ratio of Planck's constant, $h$, to the neutron mass, $m_n$. 

In [ ]:
wavelength_cut = 2.4 * sc.units.angstrom
chopper_detector_distance = 19 * sc.units.m
t_cut = sc.to_unit(wavelength_cut * chopper_detector_distance / HDM, 'us')
t_cut

The time-of-flight is then found, currently using the `np.remainder` functionality. 

In [ ]:
t_array = np.remainder((t_eto - t_cut + tau).values, tau.values)
t = sc.Variable(
    values = t_array, 
    unit=sc.units.us, 
    dims=['event'], 
    dtype=sc.dtype.float64) + t_cut + t_offset
t

Having corrected the time-of-flight, this coordinate in the `refl` object can be overwritten.

In [ ]:
refl.data.events.coords['tof'] = t

In [ ]:
refl.data.events

The corrected time-of-flight is now in the file, and it is possible to visualise a histogram of the values. 

In [ ]:
max_t = sc.max(refl.data.events.coords['tof'])
min_t = sc.min(refl.data.events.coords['tof'])
bins = sc.Variable(values=np.linspace(min_t.value, max_t.value, 100), 
                   unit=refl.data.events.coords['tof'].unit,
                   dims=['tof'])
sc.histogram(refl.event, bins).plot()

## Time-of-flight to wavelength conversion

The time-of-flight has been determined, therefore it is now possible to convert the time-of-flight to wavelength, an important step in determining $q$. 
`scipp` includes built in functionality to perform this conversion based on the instrument geometry. 
However, the `source_position` must be modified to account for the fact that the pulse is defined by the chopper. 

In [ ]:
chopper_sample_distance=15.0 * sc.units.m
refl.data.attrs["source_position"] = sc.geometry.position(0.0 * sc.units.m, 
                                                          0.0 * sc.units.m, 
                                                          -chopper_sample_distance)

This correction makes use of the definition of the sample position at $0, 0, 0$.

In [ ]:
refl.data.attrs['sample_position']

With this correction in place, the `scippneutron.convert` function can be used to find the wavelength. 

In [ ]:
refl.data

In [ ]:
refl.data = scn.convert(refl.data, origin='tof', target='wavelength', scatter=True)

Here, this is performed such that the `'tof'` coordinate is lost, this is not the cause for the `AmorData` implementation. 

Similar to the time-of-flight, it is possible to visualise the $\lambda$-values. 

In [ ]:
max_w = sc.max(refl.event.coords['wavelength'])
min_w = sc.min(refl.event.coords['wavelength'])
bins = sc.Variable(values=np.linspace(min_w.value, max_w.value, 100), 
                   unit=refl.event.coords['wavelength'].unit,
                   dims=['wavelength'])
sc.histogram(refl.event, bins).plot()

## Theta calculation and gravity correction

The Amor reflectometer scatters neutrons vertically in space from a horizontal scattering surface. 
Therefore, when the scattering angle, $\theta$, is found, it is necessary to consider the effect that gravity will have on the neutron's trajectory. 

<center>
    <img src='gravity.png' width=60%>
    <i>
        The effect of gravity for a neutron with a velocity of 20 ms<sup>-1</sup>; the solid blue line shows the trajectory of a neutron reflected from a surface under the influence of graivty, the dashed green line shows the trajectory without gravity if no correction, the solid orange line shows the true trajectory if the reflection were to occur with no gravity present.
    </i>
</center>

The figure above shows the trajectory of a neutron reflecting from a sample and being detected on the detector (the blue line). 
Initially assuming that all of the neutron are detected at the point $(0, 0)$ (the influence of beam and sample width is [considered below](#Resolution-functions)). 
During the trajectory, the neutron is acted on by the force of gravity, leading to the parabolic motion shown. 
It is clear that if $\theta$ were calculated without accounting for graivty (dashed green line), then the angle would be underestimated. 

The trajectory for the detected neutron can be found by considering the kinematic equations of motion. 
The time taken for the neutron to travel from $z=z_0$ to $z=z_d$ is, 

$$
t_d = \frac{z_d - z_0}{v_z}, 
$$

where, $v_z$ is the velocity of the neutron in the z-dimension. 
It is assumed that the velocity in this dimension is the main component of the total velocity and that this doesn't change over time. 
Therefore, we can calculate $v_z$ from the neutron's wavelength as, 

$$
v_z = \frac{h}{m_n\lambda}.
$$

This can be found as follows. 

In [ ]:
refl.data.events.coords["velocity"] = sc.to_unit(HDM / refl.data.events.coords['wavelength'], 'm/s')
refl.data.events.coords["velocity"]

It is assumed that the parabolic motion does not reach a maximum before the neutron is incident on the detector. 
Therefore, the velocity in the $y$-dimension at the time of reflection can be found, 

$$
v_y(0) = \frac{y_d - y_0 - 0.5 a t_d^2}{t_d},
$$

where $y_0$ is the initial position of the neutron, $y_0 = 0$, and $a$ is the acceleration due to gravity, $a = -g = 9.80665\;\text{ms}^{-2}$. 
Using this, the $y$-position of the neutron can be found at any time, $t$, 

$$
y(t) = y_0 + v_y(0)t + 0.5 a t^2, 
$$

or any $z$-position (with $v_y(0)$ expanded), 

$$
y(z) = y_0 + \Bigg[(z - z_0) \bigg(-\frac{a (z_d - z_0)^2}{2v_z^2} - y_0 + y_d\bigg)\Bigg]\frac{1}{z_d - z_0} + \frac{a (z - z_0)^2}{2v_z^2}.
$$

The derivative of this with respect to $z$ can then be found, 

$$
y'(z) = \bigg(\frac{-a(z_d-z_0)^2}{2v_z^2}-y_0 + y_d\bigg)\frac{1}{z_d-z_0} + \frac{a(z-z_0)}{v_z^2}, 
$$ 

This can be simplied when $z=z_0$ to, 

$$
y'(z_0) = \frac{-a (z_d-z_0)}{2v_z^2} + \frac{y_d-y_0}{z_d-z_0}.
$$

From which $\theta$ can be found,

$$
\theta = -\omega + \tan^{-1}\bigg(\frac{y'(z_0)z_d + y_0 - y'(z_0)z_0}{z_d}\bigg),
$$

where, $\omega$ is the angle of the sample relative to the laboratory horizon. 

We can show this in action, first by defining a function for the derivative above.

In [ ]:
def y_dash0(velocity, z_origin, y_origin, z_measured, y_measured):
    """
    Evaluation of the first dervative of the kinematic equations for for the trajectory of a neutron reflected from a surface.

    Args:
        velocity (:py:class:`scipp._scipp.core.VariableView`): Neutron velocity.
        z_origin (:py:class:`scipp._scipp.core.Variable`): The z-origin position for the reflected neutron.
        y_origin (:py:class:`scipp._scipp.core.Variable`): The y-origin position for the reflected neutron.
        z_measured (:py:class:`scipp._scipp.core.Variable`): The z-measured position for the reflected neutron.
        y_measured (:py:class:`scipp._scipp.core.Variable`): The y-measured position for the reflected neutron.

    Returns:
        (:py:class:`scipp._scipp.core.VariableView`): The gradient of the trajectory of the neutron at the origin position.
    """
    velocity2 = velocity * velocity
    z_diff = z_measured - z_origin
    y_diff = y_measured - y_origin
    return -0.5 * sc.norm(G_ACC) * z_diff / velocity2 + y_diff / z_diff

The angle is found by evaluating the position of each pixel with respect to the sample position. 

In [ ]:
y_measured = refl.data.coords['position'].fields.y
z_measured = refl.data.coords['position'].fields.z
y_origin = refl.data.coords['sample_position'].fields.y
z_origin = refl.data.coords['sample_position'].fields.z

The gradient and hence the angle can then be found. 

In [ ]:
y_dash = y_dash0(refl.data.bins.coords["velocity"], z_origin, y_origin, z_measured, y_measured)
intercept = y_origin - y_dash * z_origin
y_true = z_measured * y_dash + intercept
angle = sc.to_unit(sc.atan(y_true / z_measured).bins.constituents["data"], 'deg')
angle

The value of $\theta$ can then be found by accounting for the sample angle, $\omega$. 

In [ ]:
omega = 0.0 * sc.units.deg
refl.data.events.coords['theta'] = -omega + angle

This can be visualised similar to the wavelength and time-of-flight, or as a two-dimensional histogram of the intensity as a function of $\lambda$/$\theta$. 

In [ ]:
max_w = sc.max(refl.data.events.coords['wavelength'])
min_w = sc.min(refl.data.events.coords['wavelength'])
bins_w = sc.Variable(values=np.linspace(min_w.value, max_w.value, 50), 
                   unit=refl.event.coords['wavelength'].unit,
                   dims=['wavelength'])
max_t = sc.max(refl.data.events.coords['theta'])
min_t = sc.min(refl.data.events.coords['theta'])
bins_t = sc.Variable(values=np.linspace(min_t.value, max_t.value, 50), 
                   unit=refl.event.coords['theta'].unit,
                   dims=['theta'])
sc.bin(refl.event, [bins_t, bins_w]).bins.sum().plot()

## Determination of $q_z$-vector

The $\lambda$ and $\theta$ can be brought together to calculate the the wavevector, $q_z$, 

$$
q_z = \frac{4\pi \sin{\theta}}{\lambda}.
$$

This is shown below. 

In [ ]:
refl.data.events.coords['qz'] = 4. * np.pi * sc.sin(
    refl.data.events.coords['theta']) / refl.data.events.coords['wavelength']

In [ ]:
refl.data.events

We can then investigate the intensity as a function of $q_z$.

In [ ]:
max_q = 0.08 * sc.Unit('1/angstrom')
min_q = 0.008 * sc.Unit('1/angstrom')
bins_q = sc.Variable(values=np.linspace(min_q.value, max_q.value, 200), 
                   unit=refl.event.coords['qz'].unit,
                   dims=['qz'])
sc.histogram(refl.event, bins_q).plot(norm='log')

## Illumination correction

The above intensity as a function of $q_z$ fails to account for the difference in illumination as a function of $\theta$. 
This is because the size of the beam which illuminates the sample decreases with increasing $\theta$. 

<center>
    <img src='beam_size.png' width=60%>
    <i>
        The effect of $\theta$ on the beam size on the sample (blue line) and the resulting scale factor where the size of the sample is considered (orange line); where the beam size is 1 cm and the sample is 10 cm.
    </i>
</center>

Using the assumption that the beam size describes the full width at half maximum of the beam, the following function can be used to determine the scale factor necessary to account for the illumination variation as a function of $\theta$. 

In [ ]:
def illumination_correction(beam_size, sample_size, theta):
    """
    The factor by which the intensity should be multiplied to account for the
    scattering geometry, where the beam is Gaussian in shape.

    Args:
        beam_size (:py:class:`scipp._scipp.core.Variable`): Width of incident beam.
        sample_size (:py:class:`scipp._scipp.core.Variable`): Width of sample in the dimension of the beam.
        theta (:py:class:`scipp._scipp.core.Variable`): Incident angle.

    Returns:
        (:py:class:`scipp._scipp.core.Variable`): Correction factor.
    """
    beam_on_sample = beam_size / sc.sin(theta)
    fwhm_to_std = 2 * np.sqrt(2 * np.log(2))
    scale_factor = erf((sample_size / beam_on_sample * fwhm_to_std).values)
    return sc.Variable(values=scale_factor, dims=theta.dims)

This illumination correction scale factor is applied to each event as follows.

In [ ]:
refl.data.events.data /= illumination_correction(
    1 * sc.Unit('mm'), 10 * sc.Unit('mm'), refl.data.events.coords['theta'])

In [ ]:
refl.data.events.data

We can then show the reflected intensity as a function of $q_z$ as shown below.

In [ ]:
max_q = 0.08 * sc.Unit('1/angstrom')
min_q = 0.008 * sc.Unit('1/angstrom')
bins_q = sc.Variable(values=np.linspace(min_q.value, max_q.value, 200), 
                   unit=refl.event.coords['qz'].unit,
                   dims=['qz'])
sc.histogram(refl.event, bins_q).plot(norm='log')

# Resolution functions

The resolution function at Amor consists of three components:

- $\sigma \lambda/\lambda$: this is due to the double-blind chopper and depends on the distance between the two choppers, $d_{CC}$, and the distance from the second chopper to the detector $d_{CD}$, 

$$
\frac{\sigma \lambda}{\lambda} = \frac{d_{CC}}{2d_{CD}\sqrt{2\ln2}}.
$$

In [ ]:
chopper_chopper_distance=0.49 * sc.units.m
refl.data.attrs["sigma_lambda_by_lambda"] = chopper_chopper_distance / (
    refl.data.coords["position"].fields.z - refl.data.coords["source_position"].fields.z)
refl.data.attrs["sigma_lambda_by_lambda"] /= 2 * np.sqrt(2 * np.log(2))

- $\sigma \gamma/\theta$: this is to account for the spatial resolution of the detector pixels, which have a FWHM of $\Delta z \approx 2.5\;\text{mm}$ and the sample to detector distance, $d_{SD}$, 

$$
\frac{\sigma \gamma}{\theta} = \frac{1}{2\theta\sqrt{2\ln2}}\arctan{\frac{\Delta z}{d_{SD}}}.
$$

In [ ]:
spatial_resolution = 0.0025 * sc.units.m
fwhm = sc.to_unit(
    sc.atan(
        spatial_resolution / (
                refl.data.coords["position"].fields.z - 
                refl.data.coords["source_position"].fields.z)),"deg",)
sigma_gamma = fwhm / (2 * np.sqrt(2 * np.log(2)))
refl.data.attrs["sigma_gamma_by_theta"] = sigma_gamma / refl.data.bins.coords['theta']

- $\sigma \theta/\theta$: finally, this accounts for the width of the beam on the sample or the size of the sample (which ever is smaller), the FWHM is the range of possible $\theta$-values, accounting for the gravity correction discussed above and the sample/beam geometry,

$$
\frac{\sigma\theta}{\theta} = \frac{1}{2\sqrt{2\ln2}}\frac{\theta_{\text{max}} - \theta_{\text{min}}}{\theta_{\text{mid}}}
$$

In [ ]:
beam_size = 0.001 * sc.units.m
sample_size = 0.5 * sc.units.m
beam_on_sample = beam_size / sc.sin(refl.data.bins.coords['theta'])
half_beam_on_sample = (beam_on_sample / 2.0)
offset_positive = sc.geometry.position(
    refl.data.coords["sample_position"].fields.x, 
    refl.data.coords["sample_position"].fields.y, 
    refl.data.coords["sample_position"].fields.z + half_beam_on_sample)
offset_negative = sc.geometry.position(
    refl.data.coords["sample_position"].fields.x, 
    refl.data.coords["sample_position"].fields.y, 
    refl.data.coords["sample_position"].fields.z - half_beam_on_sample)
y_measured = refl.data.coords["position"].fields.y
z_measured = refl.data.coords["position"].fields.z
z_origin = offset_positive.fields.z
y_origin = offset_positive.fields.y
y_dash = y_dash0(refl.data.bins.coords["velocity"], z_origin, y_origin, z_measured, y_measured)
intercept = y_origin - y_dash * z_origin
y_true = z_measured * y_dash + intercept
angle_max = sc.to_unit(sc.atan(y_true / z_measured), 'deg')
z_origin = offset_negative.fields.z
y_origin = offset_negative.fields.y
y_dash = y_dash0(refl.data.bins.coords["velocity"], z_origin, y_origin, z_measured, y_measured)
intercept = y_origin - y_dash * z_origin
y_true = z_measured * y_dash + intercept
angle_min = sc.to_unit(sc.atan(y_true / z_measured), 'deg')
fwhm_to_std = 2 * np.sqrt(2 * np.log(2))
sigma_theta_position = (angle_max - angle_min) / fwhm_to_std
refl.data.attrs["sigma_theta_by_theta"] = sigma_theta_position / refl.data.bins.coords['theta']

These three contributors to the resolution function are combined to give the total resolution function, 

$$
\frac{\sigma q_z}{q_z} = \sqrt{\bigg(\frac{\sigma\lambda}{\lambda}\bigg)^2+\bigg(\frac{\sigma\gamma}{\theta}\bigg)^2+\bigg(\frac{\sigma\theta}{\theta}\bigg)^2}
$$

In [ ]:
refl.data.events.attrs['sigma_q_by_q'] = sc.sqrt(
    refl.data.attrs['sigma_lambda_by_lambda'] * refl.data.attrs['sigma_lambda_by_lambda']
    + refl.data.attrs['sigma_gamma_by_theta'] * refl.data.attrs['sigma_gamma_by_theta']
    + refl.data.attrs["sigma_theta_by_theta"] * refl.data.attrs["sigma_theta_by_theta"]).bins.constituents['data']

## Normalisation

The above steps are performed both on the sample of interest and a reference supermirror, using the `AmorData` class, thus enabling normalisation to be achieved. 

In [ ]:
sample = AmorData(data_file)
reference = AmorData(reference_file)

These result in the plots shown below.

In [ ]:
sc.histogram(sample.data.events, bins_q).plot(norm='log')

In [ ]:
sc.histogram(reference.data.events, bins_q).plot(norm='log')

The specification of the supermirror defines the normalisation that is used for it. 

In [ ]:
m_value = 5
supermirror_critical_edge = (0.022) * sc.Unit('1/angstrom')
supermirror_max_q = m_value * supermirror_critical_edge
supermirror_alpha = 0.25 / 0.088
normalisation = sc.ones(dims=['event'], shape=reference.data.events.data.shape)

The data array is first masked at values greater than the upper limit of the supermirror. 

In [ ]:
reference.data.events.masks['normalisation'] = reference.data.events.coords['qz'] >= supermirror_max_q

The value of the $q$-dependent normalisation, $n(q)$, is then defined as such that $n(q)=1$ for values of $q$ less then the critical edge of the supermirror and for values between this and the supermirror maximum the normalisation is, 

$$ n(q) = \frac{1}{1 - \alpha(q - c_{\text{sm}})}. $$

In [ ]:
normalisation.values[reference.data.events.coords['qz'].values < supermirror_max_q.value] = 1 / (
    1.0 - (supermirror_alpha) * (
        reference.data.events.coords['qz'].values[
            reference.data.events.coords['qz'].values < supermirror_max_q.value] - supermirror_critical_edge.value))
normalisation.values[reference.data.events.coords['qz'].values <supermirror_critical_edge.value] = 1

Applying this normalisation to the reference measurement data will return the neutron intensity as a function of $q$. 

In [ ]:
reference.data.bins.constituents['data'].data = reference.data.events.data / normalisation.astype(sc.dtype.float32)

In [ ]:
sc.histogram(reference.data.events, bins_q).plot(norm='log')

Binning this normalised description on the neutron intensity and the sample measurement in the same $q$-values allows the normalisation to be applied to the sample (note that scaling between these to measurements to account for counting time may be necessary). 

In [ ]:
binned_sample = sc.histogram(sample.data.events, bins_q)
binned_reference = sc.histogram(reference.data.events, bins_q)

In [ ]:
normalised_sample = binned_sample / binned_reference

In [ ]:
normalised_sample.plot(norm='log')